In [2]:
import syft as sy
import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torch.autograd as autograd

syft = sy 

hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")
james = sy.VirtualWorker(hook, id="james")
crypto_provider = james
torch = th


In [4]:
a.get().grad

NameError: name 'a' is not defined

In [ ]:
ones = Variable(torch.ones(a.shape))
ones = ones.send(a.location)

In [ ]:
a.backward()

In [ ]:
a.backward(b)

In [ ]:
x = th.tensor([0.,0], requires_grad=True).send(bob)
y = th.tensor([0.,0]).send(bob)

In [ ]:
x.backward(y)

In [ ]:
import torch as th
from syft import TorchHook
hook = TorchHook(th)
th.tensor([0,0])

In [ ]:
y_pred

In [ ]:
alice._objects[82398314832]

In [ ]:
t = torch.tensor([1.23567])
t.item()

In [ ]:
me = hook.local_worker

In [ ]:
t_sh = t.fix_precision().share(alice, bob)

In [ ]:
r_sh = t_sh * t_sh
r_sh

In [ ]:
#r_sh.get().float_precision()

In [ ]:
torch.mul(t_sh, r_sh)

In [ ]:
t = torch.tensor(range(-10, 10))*0.5

In [ ]:
fix_prec_tolerance_by_method = {
    "maclaurin": {3: 7 / 100, 4: 15 / 100, 5: 15 / 100}
}

for method, fix_prec_tolerance in fix_prec_tolerance_by_method.items():
    print(method)
    for prec_frac, tolerance in fix_prec_tolerance.items():
        print(prec_frac)
        t = torch.tensor(range(-10, 10)) * 0.5
        t_sh = t.fix_precision(precision_fractional=prec_frac).share(
            alice, bob, crypto_provider=james
        )
        r_sh = t_sh.sigmoid(method=method)
        r = r_sh.get().float_prec()
        t = t.sigmoid()
        diff = (r - t).abs().max()
        norm = (r + t).abs().max() / 2

        assert (diff / (tolerance * norm)) < 1

In [ ]:
t = torch.tensor([                2,                 191, 1,
                          0])
t = t.fix_prec().child
t[1: 3]

In [ ]:
t = torch.tensor(range(-10, 10))*0.5
t.sigmoid()
t_sh = t.fix_prec(precision_fractional=5)
r1 = t_sh.sigmoid().float_prec()
one = t_sh * 0 + 1
r2 = (one / (1 + (t_sh * -1).exp())).float_prec()
print(t.sigmoid())
print(r1)
print(r2)

In [ ]:
t = torch.tensor(3)
print(t.fix_prec(precision_fractional=5).exp().float_prec())
t = torch.tensor(-3)
t1 = t.fix_prec(precision_fractional=5)
(t1 * -1).exp().float_prec()

In [ ]:
m = torch.tensor([[1, 0.2], [0.4, 4.0]])
m @ m

In [ ]:
m = torch.tensor([[1, 0.2], [0.4, 4.0]])
m ** 2

In [ ]:
def inverse(self, iterations=10):
    """
    Computes an approximation of the matrix inversion using Newton-Schulz
    iterations
    """
    assert len(self.shape) >= 2, "Can't compute inverse on non-matrix"
    assert self.shape[-1] == self.shape[-2], "Must be batches of square matrices"

    #inverse = 3 * (0.5 - self).exp() + 0.003
    inverse = (0.1 * torch.eye(self.shape[-1]))

    print(inverse)

    for _ in range(iterations):
        inverse = 2 * inverse - inverse @ self @ inverse
        print(inverse)

    return inverse

In [ ]:
m = torch.tensor([[1, 0.2], [0.4, 4.0]])
m.inverse()

In [ ]:
inverse(m.fix_prec())

In [ ]:
x = m.fix_prec()
x.inverse().float_prec()

In [ ]:
m = torch.tensor([[1, 2], [3, 4.0]])
x = m.fix_prec()
y = torch.matmul(x, x).float_prec()

assert (y == torch.matmul(m, m)).all()

# with AST
m = torch.tensor([[1, 2], [3, 4.0]])
x = m.fix_prec()
y = m.fix_prec().share(bob, alice, crypto_provider=james)

z = (x @ y).get().float_prec()
z

In [ ]:
alice.clear_objects()
bob.clear_objects()

In [ ]:
x = th.tensor([2., 2., 2.])
p = x.send(bob).send(alice)

z = p + p

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
z.get()

In [ ]:
alice._objects

In [ ]:
@sy.func2plan([th.Size((3,))])
def plan_double_abs(x):
    x = x.send(bob)
    x = x + x
    x = th.abs(x)
    return x

ptr_result = plan_double_abs(th.ones(3))
assert isinstance(ptr_result.child, sy.PointerTensor)
result = ptr_result.get()
print(result)
assert torch.equal(result, th.tensor([2., 2., 2.]))

In [ ]:
big_hospital = bob
small_hospital, crypto_provider = alice, james

In [ ]:
p = nn.Parameter(torch.tensor([[0], [0], [1], [1.0]]))

In [ ]:
p.data

In [ ]:
p.data = torch.tensor([[0], [0], [1], [1.0]]) * 2

In [ ]:
p

In [ ]:
# A Toy Model
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = torch.nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc(x)
        return x

def federated():
    # A Toy Dataset
    data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1.0]])
    target = torch.tensor([[0], [0], [1], [1.0]])

    model = Net()

    model_weight = model.fc.weight.copy()

    # Training Logic
    opt = torch.optim.SGD(params=model.parameters(), lr=0.1)

    data = data.send(big_hospital)
    target = target.send(big_hospital)

    # NEW) send model to correct worker
    model.send(data.location)

    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target) ** 2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    assert (model_weight - model.get().fc.weight).sum().abs() > 1.0e-3
    
federated()

In [ ]:
import torch as th

In [ ]:
x = th.nn.Parameter(th.tensor([1., 3]))

In [ ]:
x.data = th.tensor([1., 2])

In [ ]:
x.Rdata = x.data

In [ ]:
x.Rdata = th.tensor([1., 2])

In [ ]:
th.nn.Parameter.Zdata = th.nn.Parameter.data 

In [ ]:
x.Zdata = th.tensor([1., 2])

In [ ]:
type(th.uint8)

In [ ]:
str(th.uint8)

In [ ]:
p = x.send(bob)

In [ ]:
p.get()

In [ ]:
bob._known_workers

In [ ]:
def inverse(self, iterations=20, alpha=1):
    """
    Computes an approximation of the matrix inversion using Newton-Schulz
    iterations  
    Source NASA: https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19920002505.pdf
    """
    assert len(self.shape) >= 2, "Can't compute inverse on non-matrix"
    assert self.shape[-1] == self.shape[-2], "Must be batches of square matrices"
    
    eye = torch.eye(self.shape[-1])
    inverse = alpha * self.t()

    for _ in range(iterations):
        inverse = inverse @ (2 * eye - self @ inverse)
        #print(inverse)

    return inverse

In [ ]:
n = 100
for alpha in [0.0001, 0.0002, 0.0005, 0.001, 0.002, 0.005, 0.01, 0.02]:
    success = 0
    for i in range(n):
        m = th.empty(4, 4).random_(-1000, 1000)/100
        d = (m.inverse() - inverse(m)).abs().sum()
        if d < 0.5:
            success += 1
        else:
            pass
            #print('=====', i)
            #print(m.inverse())
            #print(inverse(m))

    print(f"{alpha}\t", '='*int(success), 'o'*int(n-success), int(success/n*100), '%')

In [ ]:
th.empty(4, 4).random_(0, 10).t()

In [ ]:
m = th.tensor([[1., 2], [3, 4]])
m.inverse()

In [ ]:
inverse(m)

In [ ]:
mp = m.fix_prec(precision_fractional=3)

In [ ]:
mp.inverse().float_prec()

In [ ]:
m.inverse() 

In [ ]:
m ** -1

In [ ]:
m = th.tensor([[1., 2], [3, 4]])
iterations = 4

In [ ]:
(1 + m / 2 ** iterations).matrix_power(2 ** iterations)

In [ ]:
m.matrix_power(2)

In [ ]:
len(m.shape)

In [ ]:
m.inverse()

In [ ]:
mp = m.fix_prec(precision_fractional=5).share(alice, bob, crypto_provider=james)

In [ ]:
mp.inverse().get().float_prec()

In [ ]:
t1 = torch.tensor([1.])
t2 = torch.tensor([5.])
x1 = t1.fix_prec().share(bob, alice, crypto_provider=james)
x2 = t2.fix_prec().share(bob, alice, crypto_provider=james)

In [ ]:
t1 = 1.
t2 = torch.tensor([5.])
x1 = t1
x2 = t2.fix_prec().share(bob, alice, crypto_provider=james)
x2

In [ ]:
x1

In [ ]:
y = (x1 / x2).get().float_prec()

In [ ]:
t1 / t2

In [ ]:
t1.__div__(t2)

In [ ]:
t2.__rdiv__(t1)

In [ ]:
t1 = #
t2 = torch.tensor([5.])
t1 / t2

In [ ]:
%%time
y = (x1 / x2).get().float_prec()

In [ ]:
x = th.tensor([4., 6, 8])
x = x.fix_prec(precision_fractional=3).share(alice, bob, crypto_provider=james)

q = th.tensor([2.])
q = q.fix_prec(precision_fractional=3).share(alice, bob, crypto_provider=james)

In [ ]:
r = x / q

In [ ]:
r.get().float_prec()

In [ ]:
list(q.shape)

In [ ]:
torch.Size([1])

In [ ]:
x = th.tensor([-1.])
x = x.fix_prec(precision_fractional=3).share(alice, bob)

y = (x / 2)
print(y)
print(y.get().float_prec())

In [ ]:
import re
content = 'content """hod  '+ """s 
""" +"""s def hjleh()
""" + 'le"""' 
#content = content.replace('\n', ' ')

In [ ]:
results = re.search("def (\w+?)\(", content)
if results is not None:
    script_name = results.group(1)
else:
    script_name = f"{script_file} (looks broken)"
script_name

In [ ]:
print(content)

In [ ]:
results = re.search('"""(.*?)"""', content)
if results is not None:
    script_name = results.group(1)
else:
    script_name = f"sss (looks broken)"
print(script_name)

In [ ]:
@sy.func2plan(args_shape=[(1,)])
def my_plan(data):
    x = data * 2
    y = (x - 2) * 10
    return x + y

x = th.tensor([-1, 2, 3])
local_res = my_plan(x)

plan_ptr = my_plan.send(alice)
x_ptr = x.send(alice)
p = plan_ptr(x_ptr)
print(p)
plan_res = p.get()

assert (plan_res == local_res).all()

# delete remote object before websocket connection termination
del x_ptr

In [ ]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

        self.bias = th.tensor([1000.0])

        self.state += ["fc1", "fc2", "bias"]

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0) + self.bias

net = Net()

x = th.tensor([-1, 2.0])
local_res = net(x)
assert not net.is_built

net.build(x)

plan_ptr = net.send(alice)
x_ptr = x.send(alice)
remote_res = plan_ptr(x_ptr).get()

assert (remote_res == local_res).all()

# delete remote object before websocket connection termination
del x_ptr

In [ ]:
x11 = th.tensor([-1, 2.0]).tag("input_data")
x21 = th.tensor([-1, 2.0]).tag("input_data")

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11,))
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21,))

class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 1)

        self.bias = th.tensor([1000.0])

        self.state += ["fc1", "fc2", "bias"]

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0) + self.bias

net = Net()

# build
net.build(th.tensor([1, 2.0]))

net.send(device_1)
pointer_to_data = x11.send(device_1)#device_1.search("input_data")[0]
pointer_to_result = net(pointer_to_data)
pointer_to_result

In [ ]:
x = torch.tensor([5])
x_bin = decompose(x)[0]
print(x_bin)
moduli = 2 ** torch.arange(Q_BITS)
print(moduli)
coeffs = (torch.exp(x_bin.float()) ** moduli.float())
print(torch.prod(coeffs))

In [ ]:
2.7183 * 54.5981

In [ ]:
e = torch.exp(torch.tensor(1.))

In [ ]:
e**5

In [ ]:
 e**(1*2**2 + 0*2**1 + 1*2**0)

In [ ]:
(e**1)**(2**2) * (e**0)**(2**1) * (e**1)**(2**0)

In [ ]:
(e**(2**2) if 1 else 1)*(e**(2**1) if 0 else 1)*(e**(2**0) if 1 else 1)

In [ ]:
e**5 == (e**(2**2) if 1 else 1)*(e**(2**1) if 0 else 1)*(e**(2**0) if 1 else 1)

In [ ]:
def exp_bit(x):
    # Take a bit (0 or 1, here 0 for example)and share it in a *binary* field
    #x = torch.tensor([0])
    x_sh = x.share(alice, bob, crypto_provider=crypto_provider, field=2)
    
    # Access shares
    x0, x1 = x_sh.child.child['alice'], x_sh.child.child['bob']
    x0 = x0.float()
    x1 = x1.float()
    print(alice._objects[x0.id_at_location], bob._objects[x1.id_at_location])
    
    # Compute privately the wrap field bit, which decrypts to 1 iff x0+x1 >= 2  
    x0_sh = x0.fix_precision().share(alice, bob, crypto_provider=charlie).get()
    x1_sh = x1.fix_precision().share(alice, bob, crypto_provider=charlie).get()
    wrap_field = x0_sh * x1_sh 
    
    # Compute exp of shares
    exp_x0, exp_x1 = [torch.exp(x0), torch.exp(x1)]
    alice._objects[exp_x0.id_at_location], bob._objects[exp_x1.id_at_location]

    # Share the exp of shares
    exp_x0_sh = exp_x0.fix_precision().share(alice, bob, crypto_provider=charlie).get()
    exp_x1_sh = exp_x1.fix_precision().share(alice, bob, crypto_provider=charlie).get()

    # Apply exp(x0 + x1) =  exp(x0) *  exp(x1) formula + a wrapping correction if needed
    one = torch.tensor([1.]).fix_precision()
    inv_exp_field_size = torch.exp(-torch.tensor([2.])).fix_precision()
    exp_sh = exp_x0_sh * exp_x1_sh * (wrap_field * (inv_exp_field_size - one) + one)

    # Open and get 1.0
    return exp_sh


# Example
x = torch.tensor([0, 1, 0, 1, 0, 1, 1, 0, 0])
exp_x = exp_bit(x)
print(exp_x.get().float_prec())

## Clear text demo

In [ ]:
x = torch.tensor([5, 3, 1])
x_bin = decompose(x)

exp_x_bin = torch.exp(x_bin.float())
print(exp_x_bin)
exp_x_bin_list = exp_x_bin.unbind(dim=1)
print(exp_x_bin_list)

moduli = 2 ** torch.arange(Q_BITS)
moduli_list = map(lambda x:x.item(), moduli.unbind(dim=0))

coeffs_list = [
    xp_x_bin_item ** modulo
    for xp_x_bin_item, modulo
    in zip(exp_x_bin_list, moduli_list)
]

coeffs = torch.stack(coeffs_list).t()
print(coeffs)

torch.prod(coeffs, dim=1)

## Encrypted Demo

In [ ]:
x = torch.tensor([5, 3, 1])
x_bin = decompose(x)
print('bin', x_bin)
exp_x_bin_sh = exp_bit(x_bin)
print('exp sh', exp_x_bin_sh.child.child.virtual_get())
exp_x_bin_list = torch.unbind(exp_x_bin_sh, dim=1)

moduli = 2 ** torch.arange(Q_BITS)
moduli_list = list(map(lambda x:x.fix_precision(precision_fractional=0), moduli.unbind(dim=0)))

coeffs_list = [
    xp_x_bin_item ** modulo
    for xp_x_bin_item, modulo
    in zip(exp_x_bin_list, moduli_list)
]

exp_x_sh = coeffs_list[0]
for coeff in coeffs_list[1:]:
    exp_x_sh = exp_x_sh * coeff

#coeffs = torch.stack(coeffs_list).t()
#print('coeffs', coeffs.child.child.virtual_get())

#exp_x_sh = torch.prod(coeffs, dim=1)

print('exp_x', exp_x_sh.get().float_prec())

In [ ]:
x = torch.tensor([5, 3]).fix_precision()

torch.prod(x)

In [ ]:
# x = torch.tensor([5, 3])
x_bin = decompose(x)

exp_x_bin = torch.exp(x_bin.float())
print(exp_x_bin)
exp_x_bin_list = exp_x_bin.unbind(dim=1)
print(exp_x_bin_list)

moduli = 2 ** torch.arange(Q_BITS)
moduli_list = map(lambda x:x.item(), moduli.unbind(dim=0))

coeffs_list = [
    xp_x_bin_item ** modulo
    for xp_x_bin_item, modulo
    in zip(exp_x_bin_list, moduli_list)
]

coeffs = torch.stack(coeffs_list).t()
print(coeffs)

torch.prod(coeffs, dim=1)

In [ ]:
# Take a bit (0 or 1, here 0 for example)and share it in a *binary* field
x = torch.tensor([0])
x_sh = x.share(alice, bob, crypto_provider=crypto_provider, field=2)

In [ ]:
# Access shares
x0, x1 = x_sh.child.child['alice'], x_sh.child.child['bob']
x0 = x0.float()
x1 = x1.float()
print(alice._objects[x0.id_at_location], bob._objects[x1.id_at_location])

In [ ]:
# Compute privately the wrap field bit, which decrypts to 1 iff x0+x1 >= 2  
x0_sh = x0.fix_precision().share(alice, bob, crypto_provider=charlie).get()
x1_sh = x1.fix_precision().share(alice, bob, crypto_provider=charlie).get()
wrap_field = x0_sh * x1_sh 

In [ ]:
# Compute exp of shares
exp_x0, exp_x1 = [torch.exp(x0), torch.exp(x1)]
alice._objects[exp_x0.id_at_location], bob._objects[exp_x1.id_at_location]

In [ ]:
# Share the exp of shares
exp_x0_sh = exp_x0.fix_precision().share(alice, bob, crypto_provider=charlie).get()
exp_x1_sh = exp_x1.fix_precision().share(alice, bob, crypto_provider=charlie).get()

In [ ]:
# Apply exp(x0 + x1) =  exp(x0) *  exp(x1) formula + a wrapping correction if needed
one = torch.tensor([1.]).fix_precision()
inv_exp_field_size = torch.exp(-torch.tensor([2.])).fix_precision()
exp_sh = exp_x0_sh * exp_x1_sh * (wrap_field * (inv_exp_field_size - one) + one)

In [ ]:
# Open and get 1.0
exp_sh.get().float_prec()

In [ ]:

x = torch.tensor([[3, 4]])
y = torch.tensor([[3, 4]])
x_sh = x.share(alice, bob, crypto_provider=charlie)
y_sh = x.share(alice, bob, crypto_provider=charlie)

result = x_sh * y_sh
assert torch.equal(result.get(), x * y) 

alice.close()
time.sleep(0.1)
alice.remove_worker_from_local_worker_registry()

bob.close()
time.sleep(0.1)
bob.remove_worker_from_local_worker_registry()

charlie.close()
time.sleep(0.1)
charlie.remove_worker_from_local_worker_registry()
server.terminate()

In [ ]:
# Plaintext reference

x = torch.tensor([[1.0, 2], [1.0, 2]])
target = torch.tensor([[1.0], [1.0]])
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([-1.0]))

output = model(x)
loss = ((output - target) ** 2).sum()

loss.backward()

expected_weight_grad = model.weight.grad
expected_bias_grad = model.bias.grad

# Classic share with requires_grad

x = torch.tensor([[1.0, 2], [1.0, 2]]).fix_prec().share(bob, alice, crypto_provider=james, requires_grad=True)
target = torch.tensor([[1.0], [1.0]]).fix_prec().share(bob, alice, crypto_provider=james, requires_grad=True)
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([-1.0]))
model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)


output = model(x)
loss = ((output - target) ** 2).sum()
loss.backward()

weight_grad = model.weight.grad.get().float_precision()
bias_grad = model.bias.grad.get().float_precision()

assert (expected_weight_grad == weight_grad).all()
assert (expected_bias_grad == bias_grad).all()

# Remote (data+model) fix_prec+share with requires_grad

x = torch.tensor([[1.0, 2], [1.0, 2]]).send(charlie).fix_prec().share(bob, alice, crypto_provider=james, requires_grad=True).get()
target = torch.tensor([[1.0], [1.0]]).send(charlie).fix_prec().share(bob, alice, crypto_provider=james, requires_grad=True).get()
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([-1.0]))
model.send(charlie).fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True).get()


output = model(x)
loss = ((output - target) ** 2).sum()
loss.backward()

weight_grad = model.weight.grad.get().float_precision()
bias_grad = model.bias.grad.get().float_precision()

assert (expected_weight_grad == weight_grad).all()
assert (expected_bias_grad == bias_grad).all()

# Local fix_prec & remote data share with requires_grad

x = torch.tensor([[1.0, 2], [1.0, 2]]).fix_prec().send(charlie).share(bob, alice, crypto_provider=james, requires_grad=True).get()
target = torch.tensor([[1.0], [1.0]]).fix_prec().send(charlie).share(bob, alice, crypto_provider=james, requires_grad=True).get()
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([-1.0]))
model.fix_precision().send(charlie).share(bob, alice, crypto_provider=james, requires_grad=True).get()


output = model(x)
loss = ((output - target) ** 2).sum()
loss.backward()

weight_grad = model.weight.grad.get().float_precision()
bias_grad = model.bias.grad.get().float_precision()

assert (expected_weight_grad == weight_grad).all()
assert (expected_bias_grad == bias_grad).all()

In [ ]:
x = torch.tensor([[1.0, 2], [1.0, 2]]).fix_prec().share(bob, alice, crypto_provider=james, requires_grad=True)
target = torch.tensor([[1.0], [1.0]]).fix_prec().share(bob, alice, crypto_provider=james, requires_grad=True)
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([-1.0]))
model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)


output = model(x)
loss = ((output - target) ** 2).sum()
loss.backward()

weight_grad = model.weight.grad.get().float_precision()
bias_grad = model.bias.grad.get().float_precision()

x = torch.tensor([[1.0, 2], [1.0, 2]])
target = torch.tensor([[1.0], [1.0]])
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([-1.0]))

output = model(x)
loss = ((output - target) ** 2).sum()

loss.backward()
assert (model.weight.grad == weight_grad).all()
assert (model.bias.grad == bias_grad).all()

In [ ]:
backward_one = True
a = (
    torch.tensor([[3.0, 2], [-1, 2]], requires_grad=True)
        .fix_prec()
        .share(alice, bob, crypto_provider=james)
)
b = 2

a = sy.AutogradTensor().on(a)

a_torch = torch.tensor([[3.0, 2], [-1, 2]], requires_grad=True)
b_torch = 2

c = a / b
c_torch = a_torch / b_torch

ones = torch.ones(c.shape).fix_prec().share(alice, bob, crypto_provider=james)
ones = sy.AutogradTensor().on(ones)
c.backward(ones if backward_one else None)
c_torch.backward(torch.ones(c_torch.shape))

print(c.get().child.float_prec())
print(a.grad.get().float_prec())
print(a_torch.grad)

In [ ]:
bob._objects
(x_sh + x_sh).get()

In [ ]:
t = torch.tensor([1, 2, 3, 4.0])
ptr = t.send(james)

x = ptr.fix_prec().share(bob, alice)

y = x + x

y = y.get().get().float_prec()
assert (y == (t + t)).all()

In [ ]:
x_sh + x_sh

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn

import syft as sy
hook = sy.TorchHook(torch)
worker = sy.VirtualWorker(hook, id="alice")

class MyFunction(torch.autograd.Function):
    def forward(self, x):
        print("Custom forward called")
        return x
    
    def backward(self, grad_out):
        grad_input = grad_out.clone()
        print('Custom backward called!')
        return grad_input


x = torch.tensor([1.5], requires_grad=True)
x = x.send(worker, local_autograd=True)

y = MyFunction(x)
print(y)
y.backward()

In [ ]:
import torch
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import syft as sy
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")
import sys

alice = sy.VirtualWorker(hook, id="worker")
x = torch.tensor([1.])
x_ptr = x.send(alice)
x_fp = x_ptr.fix_prec()

In [ ]:
import torch
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import syft as sy
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")
import sys

In [ ]:
import numpy as np
jon = james

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn

import syft as sy
hook = sy.TorchHook(torch)
worker = sy.VirtualWorker(hook, id="alice")

class MyFunction(torch.autograd.Function):
    def forward(self, x):
        print("Custom forward called")
        return x
    
    def backward(self, grad_out):
        grad_input = grad_out.clone()
        print('Custom backward called!')
        return grad_input


x = torch.tensor([1.5], requires_grad=True)
x = x.send(worker, local_autograd=True)

y = MyFunction(x)
print(y)
y.backward()

In [ ]:
wks = [ sy.VirtualWorker(hook, id="w#%d" % i) for i in range(2) ]

In [ ]:
wks = [alice, bob]

In [ ]:
t = torch.zeros(3,3)
t = t.fix_prec().share(*wks, crypto_provider=james)
t = t * 2
t = t.get()
t.float_prec()

In [ ]:
x = th.tensor([1., 2., 3.]).fix_precision().send(alice)
x

In [ ]:
x = x.share(alice, bob, crypto_provider=jon)
x

In [ ]:
x.get()

In [ ]:
x_ptr = th.tensor([1., 2., 3.]).fix_precision()#.send(jon)
y_ptr = th.tensor([4., 5., 6.]).fix_precision()#.send(jon)

x_shared = x_ptr.share(alice, bob, crypto_provider=jon)#.get()
y_shared = y_ptr.share(alice, bob, crypto_provider=jon)#.get()

print(x_shared)
print(y_shared)
secure_sum = x_shared + y_shared
print(secure_sum)
# (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
# 	-> (Wrapper)>[PointerTensor | me:92050228605 -> alice:43660865053]
# 	-> (Wrapper)>[PointerTensor | me:15127770581 -> bob:67436046413]
# 	*crypto provider: jon*

secure_sum_squared = secure_sum * secure_sum
print(secure_sum_squared)
# (Wrapper)>FixedPrecisionTensor>None

In [ ]:
x_ptr = th.tensor([1., 2., 3.]).fix_precision()
y_ptr = th.tensor([4., 5., 6.]).fix_precision()

x_shared = x_ptr.share(alice, bob, crypto_provider=jon)#.get()
y_shared = y_ptr.share(alice, bob, crypto_provider=jon)#.get()

secure_sum = x_shared + y_shared
print(secure_sum)

secure_sum_squared = secure_sum * secure_sum
print(secure_sum_squared)

In [ ]:
q = 100
base = 2
prec = 3
C = 10 #base ** prec
print(q, C)

In [ ]:
def trunc(x1, x2):
    return x1/C, q - (q - x2)/C

In [ ]:
trunc(70, 70) # 40 -> 4

In [ ]:
def _trunc(x1, x2):
    if x1 > q/2:
        x1 = (x1 - q)
    if x2 > q/2:
        x2 = (x2 - q)
    x1 /= C
    x2 /= C
    return x1, x2

In [ ]:
_trunc(70, 70) # 40 -> 4

In [ ]:
def enc(x):
    return np.round(x * C) % q

def dec(x):
    if x > q/2:
        x = (x - q)
    # Convert to float
    return x / C

def add(x, y):
    z = x + y
    return z % q

def mul(x, y):
    z = (x * y) % q
    r = dec(z)
    print(r)
    return r

In [ ]:
q/(C*2)

In [ ]:
a, b = -1, 4
_a = enc(a)
_b = enc(b)
print(dec(add(_a, _b)))
print(dec(mul(_a, _b)))

In [ ]:
1025 - 8 * 8 *4

In [ ]:
a = -2
b = -3
a * b

In [ ]:
def mul(x, y):
    return ((x * y) % q)/C

In [ ]:
mul(enc(a), enc(b))

In [ ]:
def dec(x):
    if x > q/2:
        x = (x - q)
    # Convert to float
    return x / C

In [ ]:
dec(8)

In [ ]:
dec(1017)

In [ ]:
dec(mul(enc(a), enc(b)))

In [ ]:
a = 2
b = 3

In [ ]:
dec(mul(enc(a), enc(b)))

In [ ]:
a = -2
b = 1
enc(a), enc(b)

In [ ]:
(enc(a) * enc(b)) % q

In [ ]:
dec(897) / C

In [ ]:
mul(enc(a), enc(b))

In [ ]:
enc(-3*8)

In [ ]:
enc(1)

In [ ]:
def enc(x):
    return (x * C) % field
def dec(x):
    ## wrap in [0, q]
    #x = x % q
    # wrap in [-q/2, q/2]
    if x > q/(2):
        x = (x - q)
    # Convert to float
    return x / C
def add(x, y):
    return (x + y) % q
def mul(x, y):
    return np.round((x * y) / C) % q

In [ ]:
a, b = -1, -1

In [ ]:
mul(enc(a), enc(b))

In [ ]:
dec(mul(enc(a), enc(b)))

In [ ]:
x = torch.tensor([1.])
p = x.send(bob)

In [ ]:
%%time
l = []
x = torch.tensor([1.])
p = x.send(bob)
for i in range(10000):
    if hasattr(p, 'child'):
        l.append(p.child)

In [ ]:
%%time
l = []
x = torch.tensor([1.])
p = x.send(bob)
for i in range(10000):
    try:
        l.append(p.child)
    except KeyError:
        pass
        

In [ ]:
%%time
l = []
for i in range(10000):
    x = torch.tensor([1.])
    p = x.send(bob)
    if hasattr(p, 'child'):
        l.append(p.child)

In [ ]:
%%time
l = []
for i in range(10000):
    x = torch.tensor([1.])
    p = x.send(bob)
    try:
        l.append(p.child)
    except KeyError:
        pass
        

In [ ]:
dec(mul(a_fp, b_fp)) 

In [ ]:
a * (b * C + q) 

In [ ]:
a * b * C + a * q  ,   a * b * C

In [ ]:
dec((a * (b * C + q)) % q)

In [ ]:
mul(x_fp, y_fp) % (field * base ** prec)

In [ ]:
dec(mul(x_fp, y_fp)), x * y

In [ ]:
2.25 * (6.625 - field / base ** prec)

In [ ]:
x = th.tensor([1.])
x_fp = x.fix_precision()

r_fp = x_fp / 5
r = r_fp.float_precision()
print(r)
assert r_fp.float_precision() == x / 5

In [ ]:
x = th.tensor([1.])
x_fp = x.fix_precision()

r_fp = x_fp + 10
r = r_fp.float_precision()
print(r)
assert r == x + 10

r_fp = x_fp - 7
r = r_fp.float_precision()
print(r)
assert r == x - 7

r_fp = x_fp * 2
assert r_fp.float_precision() == x * 2

r_fp = x_fp / 5
assert r_fp.float_precision() == x / 5

In [ ]:
x = th.tensor([2.])
x_sh = x.fix_precision().share(alice, bob, crypto_provider=james)

r_sh = x_sh + 10
assert r_sh.get().float_prec() == x + 10

r_sh = x_sh - 7
assert r_sh.get().float_prec() == x - 7

r_sh = x_sh * 2
assert r_sh.get().float_prec() == x * 2

r_sh = x_sh / 2
assert r_sh.get().float_prec() == x / 2

In [ ]:
x = th.tensor([1.])
y = th.tensor([2.])
z = torch.mul(x, y)
z.backward()
z.grad_fn

In [ ]:
x = th.tensor([1])
x = x.share(alice, bob, crypto_provider=james).child
y = th.tensor([2])
y = y.share(alice, bob, crypto_provider=james).child

In [ ]:
z = torch.mul(x, y)

In [ ]:
z
z.virtual_get()

In [ ]:
z.grad_fn

In [ ]:
z.backward()

In [ ]:
x = th.tensor([1, 2])
x.wrap()

In [ ]:
x = th.tensor([1, 6])
xsh = x.share(alice, bob, crypto_provider=james).child

In [ ]:
xsh

In [ ]:
args = []
kwargs = {'dim':0}
self = xsh

summ = self.sum(*args, **kwargs)

 # We need to know how many input values are used for each
# output value to divide
dims_to_reduce = args if args else range(self.dim())
print(dims_to_reduce)

div = 1
for i, s in enumerate(self.shape):
    if i in dims_to_reduce:
        div *= s

z = summ // div
z.get()

In [ ]:
other_shares

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Instantiate the model
model = Net()

# The data itself doesn't matter as long as the shape is right
mock_data = torch.zeros(1, 2)

# Create a jit version of the model
traced_model = torch.jit.trace(model, mock_data)

type(traced_model)

In [ ]:
isinstance(traced_model, torch.jit.ScriptModule)

In [ ]:
@torch.jit.script
def loss_fn(real, pred):
    return ((real.float() - pred.float()) ** 2).mean()

In [ ]:
loss_fn

In [ ]:
isinstance(loss_fn, torch.jit.ScriptModule)

In [ ]:
type(loss_fn)

In [ ]:
l = [1, 2, 3]
l.drop(4)
l

In [ ]:
x = torch.Tensor([-1., -2, 3, 4])
x = x.fix_precision()
print(x)
x_priv = x.share(alice, bob, crypto_provider=james)
print(x)
x = torch.max(x)
x.get()

In [ ]:
t = torch.tensor([1., 2, 3, 4]).send(alice)

In [ ]:
t.get()

In [ ]:
t = torch.tensor([1., 2, 3, 4]).fix_precision().share(alice, bob, crypto_provider=james)

In [ ]:
torch.add(t, t)

In [ ]:
t.child.child.child.torch.nn.__module__

In [ ]:
t.child.child.child.torch.stack.__module__

In [ ]:
torch.stack([t, t])

In [ ]:
t = torch.tensor([1., 2, 3, 4]).fix_precision().share(alice, bob, crypto_provider=james)
t

In [ ]:
lr = torch.tensor([0.5]).fix_precision()

In [ ]:
lr

In [ ]:
t.child.child.child, lr.child.child

In [ ]:
t * lr

In [ ]:
t.child.child.child * lr.child.child

In [ ]:
client_copy=[0]*2;
for epoch in range(1, 2):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        if batch_idx==0:
            bobmodel=model.copy().send(data.location) # <-- NEW: send the model to the rig
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = bobmodel(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step() 
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.data))

In [ ]:
t = torch.tensor([1, 2, 3, 4]).send(bob)

In [ ]:
t.shape == torch.Size([4])

In [ ]:
t = torch.tensor([1, 2, 3, 4])
x = t.share(bob, alice, crypto_provider=james)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
ptr_to_sh = x.send(bob)
pointer = ptr_to_sh.remote_get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p = torch.tensor([1, 2, 3, 4]).send(bob)
p

In [ ]:
p.copy().move(alice).get()

In [ ]:
t = torch.tensor([1, 2, 3, 4])
x = t.share(bob, alice, crypto_provider=james)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x2 = x.child.reconstruct()
print(x2)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
t = torch.tensor([1, 2, 3, 4])
x = t.share(bob, alice, crypto_provider=james)
print(x)
y = (x * x)

#z = y.get()

#assert (y == (t * t)).all() 72265325022

In [ ]:
x = torch.tensor([1, 2, 3, 4, 5]).send(bob)
x.move(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    @sy.method2plan
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)
    
    def your(self, x):
        return self.forward(self, x)

In [ ]:
import torch

class MyModule(torch.jit.ScriptModule):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    @torch.jit.script_method
    def forward(self, input):
        if bool(input.sum() > 0):
            output = self.weight.mv(input)
        else:
            output = self.weight + input
        return output

my_script_module = MyModule(2, 3)

In [ ]:
my_script_module.save("playground_model.pt")

In [ ]:
my_script_module.graph

In [ ]:
import torch
import torch.nn as nn
import syft as sy
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")
import sys

In [ ]:

a = torch.tensor([3, 2., 0], requires_grad=True)
b = torch.tensor([1., 2, 3], requires_grad=True)

a = a.fix_precision().share(alice, bob, local_autograd=True, crypto_provider=james)
b = b.fix_precision().share(alice, bob, local_autograd=True, crypto_provider=james)

c = a + b

#c.child.backward

In [ ]:
a

In [ ]:
a = torch.tensor([3, 2., 0], requires_grad=True)
a = a.send(alice, local_autograd=True)

In [ ]:
a

In [ ]:
a.child.grad_fn #.grad_fn = 1

In [ ]:

a = torch.tensor([3, 2.0, 0], requires_grad=True)
b = torch.tensor([1, 2.0, 3], requires_grad=True)

a = a.send(alice, local_autograd=True)
b = b.send(alice, local_autograd=True)

a_torch = torch.tensor([3, 2.0, 0], requires_grad=True)
b_torch = torch.tensor([1, 2.0, 3], requires_grad=True)

c = a + b
c_torch = a_torch + b_torch

c.backward(torch.ones(c.shape).send(alice))
c_torch.backward(torch.ones(c_torch.shape))

assert (a.grad.get() == a_torch.grad).all()
assert (b.grad.get() == b_torch.grad).all()

In [ ]:
workers = {'bob': bob, 'alice': alice}

In [ ]:
# create tensor
x = torch.Tensor([1, 2])

# send tensor to bob and then pointer to alice
x_ptr = x.send(workers["bob"])
x_ptr_ptr = x_ptr.send(workers["alice"])

# ensure bob has tensor
assert x.id in workers["bob"]._objects

# delete pointer to pointer to tensor, which should automatically
# garbage collect the remote object on Bob's machine

print(sys.getrefcount(workers["bob"]._objects[x.id]))
del x_ptr_ptr

# ensure bob's object was garbage collected
assert x.id not in workers["bob"]._objects

In [ ]:
# create tensor
x = torch.Tensor([1, 2])

# send tensor to bob and then pointer to alice
x_ptr = x.send(bob)

print('<<')
print(sys.getrefcount(bob._objects[x_ptr.id_at_location]))
print('>>')
x_ptr_ptr = x_ptr.send(alice)

# ensure bob has tensor
assert x.id in bob._objects

# delete pointer to pointer to tensor, which should automatically
# garbage collect the remote object on Bob's machine

print(sys.getrefcount(alice._objects[x_ptr_ptr.id_at_location]))
print(sys.getrefcount(bob._objects[x.id]))
del x_ptr_ptr

# ensure bob's object was garbage collected
assert x.id not in bob._objects

In [ ]:
model = nn.Linear(2, 1)
model.weight = nn.Parameter(model.weight * 100)
model.weight

In [ ]:
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
#model.bias = nn.Parameter(torch.tensor([[1.0]]))
print(model.weight)
model.fix_precision().share(bob, alice, crypto_provider=james)
model.weight.child.child.child.virtual_get()

In [ ]:
p = nn.Parameter(torch.tensor([[-1.0, 2]]))
o = p.sum()
o.backward()
p.grad -= p.grad
p

In [ ]:
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
#for p in model.parameters():
#    p.fix_precision_().share_(bob, alice, crypto_provider=james)
model.fix_precision().share(bob, alice, crypto_provider=james)

print(model.weight.child.child.child.virtual_get())

In [ ]:
p = nn.Parameter(torch.tensor([[-1.0, 2]]))
p.fix_precision_().share_(bob, alice, crypto_provider=james)
print(p.child.child.child.virtual_get())

In [ ]:
model.weight.child.child.child.virtual_get(), model.bias.child.child.child.virtual_get()

In [ ]:
t = torch.tensor([[1.0, 2]])
x = t.fix_prec().share(bob, alice, crypto_provider=james)
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([[-1.0]]))
model.fix_precision().share(bob, alice, crypto_provider=james)

y = model(x)

back = y.get().float_prec()
print(back)
assert back == torch.tensor([[2.0]])

In [ ]:
t = torch.tensor([[1.0, 2]])
x = t
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-1.0, 2]]))
model.bias = nn.Parameter(torch.tensor([[-1.0]]))

y = model(x)

back = y
print(back)

In [ ]:

tensor = torch.tensor([1, 2, 3, 4.0])
ptr = tensor.send(bob)
r_ptr = torch.split(ptr, 2)
assert (r_ptr[0].get() == torch.tensor([1, 2.0])).all()

tensor = torch.tensor([1, 2, 3, 4.0])
ptr = tensor.send(bob)
max_value, argmax_idx = torch.max(ptr, 0)